# SPY Spreadsheet

In [1]:
%matplotlib inline

Use future imports for python 3.0 forward compatibility

In [2]:
from __future__ import print_function
from __future__ import unicode_literals
from __future__ import division
from __future__ import absolute_import

Other imports

In [3]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import datetime
from talib.abstract import *
import pinkfish as pf
import itable

# format price data
pd.options.display.float_format = '{:0,.2f}'.format

# set size of inline plots
matplotlib.rcParams['figure.figsize'] = [14, 10]

Some global data

In [4]:
symbol = 'SPY'
capital = 10000
start = datetime.datetime(1900, 1, 1)
end = datetime.datetime.now()

Fetch symbol data from internet; do not use local cache. 

In [5]:
ts = pf.fetch_timeseries(symbol, use_cache=False)

In [6]:
ts.tail()

,high,low,open,close,volume,adj_close
date,,,,,,
2019-05-09,287.33,283.30,285.23,286.66,"103,471,100.00",286.66
2019-05-10,288.94,282.30,285.62,288.10,"112,429,300.00",288.10
2019-05-13,283.49,279.93,282.42,280.86,"127,290,500.00",280.86
2019-05-14,285.10,281.85,281.99,283.40,"77,003,200.00",283.40
2019-05-15,285.77,281.36,281.59,285.06,"73,772,500.00",285.06


Select timeseries between start and end.  Back adjust prices relative to adj_close for dividends and splits.

In [7]:
ts = pf.select_tradeperiod(ts, start, end, use_adj=True)

In [8]:
ts.head()

,high,low,open,close,volume,adj_close
date,,,,,,
1993-01-29,26.86,26.72,26.86,26.84,"1,003,200.00",26.84
1993-02-01,27.03,26.86,26.86,27.03,"480,500.00",27.03
1993-02-02,27.10,26.95,27.01,27.08,"201,300.00",27.08
1993-02-03,27.39,27.10,27.12,27.37,"529,400.00",27.37
1993-02-04,27.54,27.16,27.47,27.49,"531,500.00",27.49


Add technical indicator: 200 day MA

In [9]:
sma200 = SMA(ts, timeperiod=200)
ts['sma200'] = sma200

In [10]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200
date,,,,,,,
2019-05-09,287.33,283.30,285.23,286.66,"103,471,100.00",286.66,275.18
2019-05-10,288.94,282.30,285.62,288.10,"112,429,300.00",288.10,275.23
2019-05-13,283.49,279.93,282.42,280.86,"127,290,500.00",280.86,275.24
2019-05-14,285.10,281.85,281.99,283.40,"77,003,200.00",283.40,275.26
2019-05-15,285.77,281.36,281.59,285.06,"73,772,500.00",285.06,275.30


Add technical indicator: ATR

In [11]:
atr = ATR(ts, timeperiod=14)
ts['atr'] = atr

In [12]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr
date,,,,,,,,
2019-05-09,287.33,283.30,285.23,286.66,"103,471,100.00",286.66,275.18,2.88
2019-05-10,288.94,282.30,285.62,288.10,"112,429,300.00",288.10,275.23,3.14
2019-05-13,283.49,279.93,282.42,280.86,"127,290,500.00",280.86,275.24,3.50
2019-05-14,285.10,281.85,281.99,283.40,"77,003,200.00",283.40,275.26,3.56
2019-05-15,285.77,281.36,281.59,285.06,"73,772,500.00",285.06,275.30,3.62


Add technical indicator: 5 day high, and 5 day low

In [13]:
high5 = pd.Series(ts.high).rolling(window=5).max()
low5 = pd.Series(ts.low).rolling(window=5).min()
ts['high5'] = high5
ts['low5'] = low5

In [14]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5
date,,,,,,,,,,
2019-05-09,287.33,283.30,285.23,286.66,"103,471,100.00",286.66,275.18,2.88,294.34,283.30
2019-05-10,288.94,282.30,285.62,288.10,"112,429,300.00",288.10,275.23,3.14,293.31,282.30
2019-05-13,283.49,279.93,282.42,280.86,"127,290,500.00",280.86,275.24,3.50,290.81,279.93
2019-05-14,285.10,281.85,281.99,283.40,"77,003,200.00",283.40,275.26,3.56,289.43,279.93
2019-05-15,285.77,281.36,281.59,285.06,"73,772,500.00",285.06,275.30,3.62,288.94,279.93


Add technical indicator: RSI, and 2-period cumulative RSI

In [15]:
rsi2 = RSI(ts, timeperiod=2)
ts['rsi2'] = rsi2

c2rsi2 = pd.Series(ts.rsi2).rolling(window=2).sum()
ts['c2rsi2'] = c2rsi2

In [16]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2
date,,,,,,,,,,,,
2019-05-09,287.33,283.30,285.23,286.66,"103,471,100.00",286.66,275.18,2.88,294.34,283.30,6.88,17.05
2019-05-10,288.94,282.30,285.62,288.10,"112,429,300.00",288.10,275.23,3.14,293.31,282.30,55.09,61.96
2019-05-13,283.49,279.93,282.42,280.86,"127,290,500.00",280.86,275.24,3.50,290.81,279.93,8.88,63.96
2019-05-14,285.10,281.85,281.99,283.40,"77,003,200.00",283.40,275.26,3.56,289.43,279.93,42.64,51.52
2019-05-15,285.77,281.36,281.59,285.06,"73,772,500.00",285.06,275.30,3.62,288.94,279.93,61.35,103.99


Add technical indicator: Midpoint

In [17]:
mp = (ts.high + ts.low)/2
ts['mp'] = mp

In [18]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp
date,,,,,,,,,,,,,
2019-05-09,287.33,283.30,285.23,286.66,"103,471,100.00",286.66,275.18,2.88,294.34,283.30,6.88,17.05,285.31
2019-05-10,288.94,282.30,285.62,288.10,"112,429,300.00",288.10,275.23,3.14,293.31,282.30,55.09,61.96,285.62
2019-05-13,283.49,279.93,282.42,280.86,"127,290,500.00",280.86,275.24,3.50,290.81,279.93,8.88,63.96,281.71
2019-05-14,285.10,281.85,281.99,283.40,"77,003,200.00",283.40,275.26,3.56,289.43,279.93,42.64,51.52,283.48
2019-05-15,285.77,281.36,281.59,285.06,"73,772,500.00",285.06,275.30,3.62,288.94,279.93,61.35,103.99,283.56


Add technical indicator: SMA10 of midpoint

In [19]:
sma10mp = pd.Series(ts.mp).rolling(window=10).mean()
ts['sma10mp'] = sma10mp

In [20]:
ts.head(10)

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp
date,,,,,,,,,,,,,,
1993-01-29,26.86,26.72,26.86,26.84,"1,003,200.00",26.84,nan,nan,nan,nan,nan,nan,26.79,nan
1993-02-01,27.03,26.86,26.86,27.03,"480,500.00",27.03,nan,nan,nan,nan,nan,nan,26.94,nan
1993-02-02,27.10,26.95,27.01,27.08,"201,300.00",27.08,nan,nan,nan,nan,100.00,nan,27.03,nan
1993-02-03,27.39,27.10,27.12,27.37,"529,400.00",27.37,nan,nan,nan,nan,100.00,200.00,27.25,nan
1993-02-04,27.54,27.16,27.47,27.49,"531,500.00",27.49,nan,nan,27.54,26.72,100.00,200.00,27.35,nan
1993-02-05,27.52,27.31,27.47,27.47,"492,100.00",27.47,nan,nan,27.54,26.86,89.31,189.31,27.42,nan
1993-02-08,27.56,27.43,27.47,27.47,"596,100.00",27.47,nan,nan,27.56,26.95,89.31,178.63,27.50,nan
1993-02-09,27.37,27.22,27.37,27.28,"122,100.00",27.28,nan,nan,27.56,27.10,16.96,106.28,27.29,nan
1993-02-10,27.33,27.20,27.28,27.31,"379,600.00",27.31,nan,nan,27.56,27.16,37.29,54.25,27.27,nan


Add technical indicator: Standard Deviation

In [21]:
sd = pd.Series(ts.mp).rolling(window=10).std()
ts['sd'] = sd

In [22]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp,sd
date,,,,,,,,,,,,,,,
2019-05-09,287.33,283.30,285.23,286.66,"103,471,100.00",286.66,275.18,2.88,294.34,283.30,6.88,17.05,285.31,290.96,2.82
2019-05-10,288.94,282.30,285.62,288.10,"112,429,300.00",288.10,275.23,3.14,293.31,282.30,55.09,61.96,285.62,290.29,3.23
2019-05-13,283.49,279.93,282.42,280.86,"127,290,500.00",280.86,275.24,3.50,290.81,279.93,8.88,63.96,281.71,289.06,3.93
2019-05-14,285.10,281.85,281.99,283.40,"77,003,200.00",283.40,275.26,3.56,289.43,279.93,42.64,51.52,283.48,288.10,4.01
2019-05-15,285.77,281.36,281.59,285.06,"73,772,500.00",285.06,275.30,3.62,288.94,279.93,61.35,103.99,283.56,287.12,3.76


In [23]:
upper = ts.sma10mp + ts.sd*2
lower = ts.sma10mp - ts.sd*2
ts['upper'] = upper
ts['lower'] = lower

In [24]:
ts.tail()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp,sd,upper,lower
date,,,,,,,,,,,,,,,,,
2019-05-09,287.33,283.30,285.23,286.66,"103,471,100.00",286.66,275.18,2.88,294.34,283.30,6.88,17.05,285.31,290.96,2.82,296.60,285.32
2019-05-10,288.94,282.30,285.62,288.10,"112,429,300.00",288.10,275.23,3.14,293.31,282.30,55.09,61.96,285.62,290.29,3.23,296.74,283.84
2019-05-13,283.49,279.93,282.42,280.86,"127,290,500.00",280.86,275.24,3.50,290.81,279.93,8.88,63.96,281.71,289.06,3.93,296.92,281.21
2019-05-14,285.10,281.85,281.99,283.40,"77,003,200.00",283.40,275.26,3.56,289.43,279.93,42.64,51.52,283.48,288.10,4.01,296.11,280.09
2019-05-15,285.77,281.36,281.59,285.06,"73,772,500.00",285.06,275.30,3.62,288.94,279.93,61.35,103.99,283.56,287.12,3.76,294.64,279.59


Select a smaller time from for use with itable

In [25]:
df = ts['2015-01-01':]

In [26]:
df.head()

,high,low,open,close,volume,adj_close,sma200,atr,high5,low5,rsi2,c2rsi2,mp,sma10mp,sd,upper,lower
date,,,,,,,,,,,,,,,,,
2015-01-02,190.19,187.70,189.73,188.85,"121,465,900.00",188.85,178.40,2.10,192.11,187.70,6.95,14.45,188.94,190.66,0.89,192.44,188.88
2015-01-05,187.88,185.10,187.69,185.44,"169,632,600.00",185.44,178.48,2.22,192.11,185.10,1.19,8.14,186.49,190.25,1.59,193.44,187.06
2015-01-06,186.36,182.81,185.78,183.69,"209,151,400.00",183.69,178.56,2.32,191.55,182.81,0.64,1.83,184.59,189.73,2.40,194.54,184.92
2015-01-07,186.36,184.67,185.17,185.98,"125,346,700.00",185.98,178.65,2.34,191.39,182.81,54.91,55.56,185.52,189.25,2.73,194.72,183.79
2015-01-08,189.52,187.53,187.55,189.28,"147,217,800.00",189.28,178.75,2.43,190.19,182.81,82.49,137.41,188.53,189.00,2.66,194.33,183.67


Use itable to format the spreadsheet.  New 5 day high has blue highlight; new 5 day low has red highlight.

In [27]:
pt = itable.PrettyTable(df, tstyle=itable.TableStyle(theme="theme1"), center=True, header_row=True, rpt_header=20)

pt.update_col_header_style(format_function=lambda x: x.upper(), text_align='right')
pt.update_row_header_style(format_function=lambda x: pd.to_datetime(str(x)).strftime('%Y/%m/%d'), text_align='right')

for col in range(pt.num_cols):
    if pt.df.columns[col] == 'volume':
        pt.update_cell_style(cols=[col], format_function=lambda x: format(x, '.0f'), text_align='right')
    else:
        pt.update_cell_style(cols=[col], format_function=lambda x: format(x, '.2f'), text_align='right')

for row in range(pt.num_rows):
    if row == 0:
        continue
    if (pt.df['high5'][row] == pt.df['high'][row]) and \
       (pt.df['high5'][row] > pt.df['high'][row-1]):
        col = df.columns.get_loc("high5")    
        pt.update_cell_style(rows=[row], cols=[col], color='blue')
    if (pt.df['low5'][row] == pt.df['low'][row]) and \
       (pt.df['low5'][row] < pt.df['low'][row-1]):
        col = df.columns.get_loc("low5")
        pt.update_cell_style(rows=[row], cols=[col], color='maroon')          

In [28]:
pt

,HIGH,LOW,OPEN,CLOSE,VOLUME,ADJ_CLOSE,SMA200,ATR,HIGH5,LOW5,RSI2,C2RSI2,MP,SMA10MP,SD,UPPER,LOWER
2015/01/02,190.19,187.70,189.73,188.85,121465900,188.85,178.40,2.10,192.11,187.70,6.95,14.45,188.94,190.66,0.89,192.44,188.88
2015/01/05,187.88,185.10,187.69,185.44,169632600,185.44,178.48,2.22,192.11,185.10,1.19,8.14,186.49,190.25,1.59,193.44,187.06
2015/01/06,186.36,182.81,185.78,183.69,209151400,183.69,178.56,2.32,191.55,182.81,0.64,1.83,184.59,189.73,2.40,194.54,184.92
2015/01/07,186.36,184.67,185.17,185.98,125346700,185.98,178.65,2.34,191.39,182.81,54.91,55.56,185.52,189.25,2.73,194.72,183.79
2015/01/08,189.52,187.53,187.55,189.28,147217800,189.28,178.75,2.43,190.19,182.81,82.49,137.41,188.53,189.00,2.66,194.33,183.67
2015/01/09,189.76,187.09,189.74,187.77,150812300,187.77,178.85,2.44,189.76,182.81,52.80,135.29,188.42,188.72,2.55,193.81,183.62
2015/01/12,188.09,185.63,187.91,186.30,144396100,186.30,178.95,2.45,189.76,182.81,31.10,83.90,186.86,188.23,2.37,192.97,183.50
2015/01/13,188.90,184.33,187.65,185.77,214553300,185.77,179.04,2.60,189.76,184.33,24.05,55.15,186.61,187.72,2.06,191.85,183.60
2015/01/14,184.87,182.55,183.54,184.65,192991100,184.65,179.11,2.64,189.76,182.55,12.21,36.26,183.71,186.98,2.03,191.03,182.92
2015/01/15,185.71,182.83,185.36,182.96,176613900,182.96,179.17,2.66,189.76,182.55,4.91,17.13,184.27,186.39,1.86,190.12,182.67
